In [1]:
from langchain.chat_models import init_chat_model
from langchain_postgres import PGVector
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.tools import tool
from dotenv import load_dotenv
import os
import getpass

load_dotenv()

True

In [2]:
if not os.environ.get("GOOGLE_API_KEY"):
  print("Please set the GOOGLE_API_KEY environment variable.")

model = init_chat_model("google_genai:gemini-2.5-flash")

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [3]:
if not os.environ.get("SUPABASE_CONNECTION_STRING"):
    os.environ["SUPABASE_CONNECTION_STRING"] = getpass.getpass("Enter connection string for Supabase: ")

vector_store = PGVector(
    embeddings=embeddings,
    collection_name="lilianweng_blog",
    connection=os.environ["SUPABASE_CONNECTION_STRING"],
)

In [4]:
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [5]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [6]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    last_message = event["messages"][-1]
    last_message.pretty_print()

c:\Users\rlarb\Developer\hai5016-rag\.venv\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.


Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  retrieve_context (678cd1d7-1a88-4675-b1c6-effac7511fda)
 Call ID: 678cd1d7-1a88-4675-b1c6-effac7511fda
  Args:
    query: standard method for Task Decomposition


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


...

================================== Ai Message ==================================
Tool Calls:
  retrieve_context (fcd2048c-acb3-48de-97d3-792ec1280f6a)
 Call ID: fcd2048c-acb3-48de-97d3-792ec1280f6a
  Args:
    query: Task Decomposition methods


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================= Tool Message =================================
Name: retrieve_context




Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

[{'type': 'text', 'text': 'I couldn\'t find information on a "standard method for Task Decomposition" in the available context. Therefore, I cannot identify common extensions of that method either.', 'extras': {'signature': 'CoQEAXLI2nyEGvDL4R9RSoU6S6Srd/xr1mdafDxZt8snkWwyQH1bwP/0po6At9cbgWzGYYVgXjcjtfhnew6MPJ78gTtKDHbB404VTBo79/auSLnWqU7TDnuqLaC0Pmb75bUCSZMWMImVwNZSRAJ/oGyprEm1zF6wkStX8aDBkVJ29NBjKcV2hEl+0l29hsA6wTQS0Y6Qx/IY8YeA3dFfxV2DvcYqG7ZUJv7IeE4tb8/PDRt+KYFbV3QBGPj9hA8OFdqRG7Kck1DB6Hyy9OrWIM14sU48C6HuyPMntaOLjCfrEw/woxj5Rv+XPrWkyOHsyYkENIl8oLG+inQuegSUijgSeMk7/IQ1SBsNpTt5rUjfkoOn6tIDxjMsmV/KaAx4PJIjen7yALjXb/tVpp8o8C1D7sUmUBi3OixIwEfunM+Ywl8hJs2ZhDMPnLQzidcrn03KZHkS+vdw/0qKQvFtcAzqS4ZKSGziU15bgNC1z8RW2WzO85uaW4MFdiZF9CdOiD/nYJnC6pJ7dy/XZk2H+RPcixmdGfdSJrbimgyeH2r1lIUi6DohQxIDotVYbB1G5XjDJiZLMYPPwh9Kq/Jd5elzyEU2bhgA1XzTgrj5C87zConQQrZEhG1Rn97onfb1EkryvRmER1tFw/ZI/BBdAIaVuk9jTnSBpkaL02Ry65zBpiRcrudL'

In [7]:
# Render the agent's last message as Markdown in the Jupyter output
from IPython.display import Markdown, display
display(Markdown(last_message.text))

I couldn't find information on a "standard method for Task Decomposition" in the available context. Therefore, I cannot identify common extensions of that method either.

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
